# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import pyodbc
import os
import warnings

# Constants

In [2]:
# Define the years for each dataframe
years = [2017, 2018, 2019, 2021, 2022]

In [3]:
temp_df = pd.read_csv('../../data/SRC2017/ARE2017.csv')
if 'PER_PROF' in temp_df.columns:
    temp_df = temp_df.drop('PER_PROF', axis=1)

if 'NUM_PROF' in temp_df.columns:
    temp_df = temp_df.drop('NUM_PROF', axis=1)

temp_df["PER_PROF"] = pd.to_numeric(temp_df["PER_LEVEL5"], errors='coerce') + \
                     pd.to_numeric(temp_df["PER_LEVEL4"], errors='coerce') + \
                     pd.to_numeric(temp_df["PER_LEVEL3"], errors='coerce')

temp_df["NUM_PROF"] = pd.to_numeric(temp_df["NUM_LEVEL5"], errors='coerce') + \
                     pd.to_numeric(temp_df["NUM_LEVEL4"], errors='coerce') + \
                     pd.to_numeric(temp_df["NUM_LEVEL3"], errors='coerce')

temp_df.to_csv('../../data/SRC2017/ARE2017.csv', index=False)

# Intialize Useful Functions

In [4]:
def safe_convert(val):
    try:
        return int(val)
    except ValueError:
        print(f"Value {val} can't be converted to int")
        return None
    
def import_mdb(MDBs, DRV, PWD, NAMES):
    
    databases = {}
    
    for MDB, NAME in zip(MDBs, NAMES):
        # connect to db
        con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB,PWD))
        cur = con.cursor()

        # List all tables in the database
        tables = list(map(lambda t: t.table_name, con.cursor().tables(tableType='TABLE')))

        # Initialize an empty dictionary to hold your dataframes and databases
        database = {}

        # Try to read each table one by one
        for table in tables:
            try:
                df = pd.read_sql(f'SELECT * FROM [{table}]', con)  # enclose table name in brackets
                database[table] = df
                print(f"Successfully read table: {table}")
            except Exception as e:
                print(f"Failed to read table: {table}")
                print(f"Error: {e}")
        databases[NAME] = database
        
    return databases

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', 'pandas only support SQLAlchemy connectable.*')
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

# Import Main Data

In [5]:
# Main Data Filepaths WINDOWS
# MAIN_PATH = [
#                '../data/SRC2019/SRC2019.mdb;',
#                '../data/SRC2021/SRC2021.mdb;',
#                '../data/SRC2022/SRC2022.mdb;',
#               ]

# Main Data Filepaths MAC
MAIN_PATH = [
               '../../data/SRC2017/ARE2017.csv',
               '../../data/SRC2018/ARE2018.csv',
               '../../data/SRC2019/ARE2019.csv',
               '../../data/SRC2021/ARE2021.csv',
               '../../data/SRC2022/ARE2022.CSV',
              ]
MAIN_NAMES = [
               'MAIN2017',
               'MAIN2018',
               'MAIN2019',
               'MAIN2021',
               'MAIN2022',
              ]

main_data = {}
for name, filepath in zip(MAIN_NAMES, MAIN_PATH):
    main_data[name] = {'Annual Regents Exams':pd.read_csv(filepath, thousands=',')}

# Main data Windows
# main_data = import_mdb(MAIN_PATH, DRV, PWD, MAIN_NAMES)

# Import Enrollment Data

In [6]:
# DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'
# PWD = 'pw'

# # Enroll Data Filepaths
# ENROLL_PATH = [
#                '../data/enrollment_2019/ENROLL2019.mdb;',
#                '../data/enrollment_2021/ENROLL2021.mdb;',
#                '../data/enrollment_2022/ENROLL2022.mdb;',
#               ]
# ENROLL_NAMES = [
#                'ENROLL2019',
#                'ENROLL2021',
#                'ENROLL2022',
#               ]


# enroll_data = import_mdb(ENROLL_PATH, DRV, PWD, ENROLL_NAMES)

# Import Dropout Rates

In [7]:
data22 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2022.csv", thousands=',')
data21 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2021.csv", thousands=',')
data19 = pd.read_csv("../../data/raw_data/GRAD_RATE_AND_OUTCOMES_2019.csv", thousands=',')

dropout_dfs = [data19, data21, data22]

# Combine Dropout Data

In [8]:
# Remove districts, only keep schools
for i, df in enumerate(dropout_dfs):
    dropout_dfs[i] = df[df['aggregation_type'] == 'School']

#   Only keep schools which are present in all years    #
#########################################################

# Convert the 'ID' column of each DataFrame to a set
set1 = set(dropout_dfs[0]['aggregation_code'])
set2 = set(dropout_dfs[1]['aggregation_code'])
set3 = set(dropout_dfs[2]['aggregation_code'])

# Find the intersection of all 4 sets - i.e., the common IDs
common_ids = set1 & set2 & set3

# Filter each DataFrame to only include rows with a common ID
for i, df in enumerate(dropout_dfs):
    dropout_dfs[i] = df[df['aggregation_code'].isin(common_ids)]
    
###########################################################

common_ids = set(df['aggregation_code'])

# Initialize a list to store the updated dataframes
updated_dfs = []

# Iterate over the dropout dataframes and the years together
for year, df in zip(years, dropout_dfs):
    # Add a new column 'year' to the dataframe
    df['year'] = year
    # Append the updated dataframe to the list
    updated_dfs.append(df)

# Concatenate the updated dataframes together
dropout_df = pd.concat(updated_dfs)

# drop disttricts from the dataframe
dropout_df = dropout_df[~dropout_df['aggregation_code'].astype(str).str.endswith('0000.0')]

# Reset the index of the combined dataframe
dropout_df = dropout_df.reset_index(drop=True)

common_ids = set(dropout_df['aggregation_code'])

In [9]:
# Define a function to check if 'comparison' year is in 'membership_desc'
def check_year_in_desc(row):
    return str(row['comparison']) in row['membership_desc'] and "August" not in row['membership_desc']

dropout_df = dropout_df[dropout_df['subgroup_name'] == 'All Students']
dropout_df['report_school_year'] = dropout_df['report_school_year'].apply(lambda x: int(str(x).split('-')[1]))
dropout_df['report_school_year'] = dropout_df['report_school_year'].apply(lambda x: x + 2000 if x < 100 else x)

# Convert the 'report_school_year' to int and subtract 4
dropout_df['comparison'] = dropout_df['report_school_year'] - 4

# Apply the function to each row of dropout_df
dropout_df = dropout_df[dropout_df.apply(check_year_in_desc, axis=1)]

# Combine Main Data

### Remove Districts, Keep Schools Common Across Databases

In [10]:
common_entity_ids = set(main_data['MAIN2019']['Annual Regents Exams']['ENTITY_CD']) 

for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    current_data['ENTITY_CD'] = current_data['ENTITY_CD'].apply(safe_convert)
    current_data = current_data[~current_data['ENTITY_CD'].astype(str).str.endswith('0000')]

    common_entity_ids = set(current_data['ENTITY_CD']) & common_ids
    
for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    main_data[database]['Annual Regents Exams'] = current_data[current_data['ENTITY_CD'].isin(common_ids)]

In [11]:
for database in main_data:
    current_data = main_data[database]['Annual Regents Exams']
    main_data[database] = {subject: current_data[current_data['SUBJECT'] == subject] for subject in current_data['SUBJECT'].unique()}

In [12]:
old_to_new = {
    'REG_PHYS_PS':'Regents Phy Set/Physics',
    'REG_NF_GLHIST':'Regents NF Global History',
    'REG_COMENG':'Regents Common Core English Language Art', 
    'REG_ESCI_PS':'Regents Phy Set/Earth Sci',
    'REG_CHEM_PS':'Regents Phy Set/Chemistry', 
    'REG_COMALG1':'Regents Common Core Algebra I', 
    'REG_COMGEOM':'Regents Common Core Geometry', 
    'REG_LENV':'Regents Living Environment',
    'REG_USHG_RV':"Regents US History&Gov't"
}

new_to_old = {}

for key in old_to_new:
    new_to_old[old_to_new[key]] = key

tests = (set(old_to_new[test] for test in old_to_new) 
         & set(test for test in main_data['MAIN2021'])
         & set(test for test in main_data['MAIN2022']))

### Calculate Demographic Percentages

In [13]:
for year, database in zip(years, MAIN_NAMES):
    current_df = None
    current_df = (main_data[database]['Regents Common Core Algebra I'] 
                  if 'Regents Common Core Algebra I' in main_data[database]
                else main_data[database][new_to_old['Regents Common Core Algebra I']])

    # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
    total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == year)][['ENTITY_CD', 'TESTED', 'YEAR']]
    print("checkpoint 1")

    # Merge the total students for 'All Students' back into the original DataFrame
    current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))
    print("checkpoint 2")

    # List of subgroups of interest
    KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

    # List to store DataFrames
    df_list = []
    columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
    columns = columns.split()

    # Get ENTITY_NAME for 'All Students' subgroup
    multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]
    print("checkpoint 3")

    # Loop over each subgroup and calculate percentage
    for subgroup in KEPT_SUBGROUPS:
        temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == year)].copy()
        subgroup = subgroup.upper()
        temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
        temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
        df_list.append(temp_df)
    print("checkpoint 4")

    # Merge all DataFrames on ENTITY_CD
    result_df = multiple_df
    for temp_df in df_list:
        result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    print("checkpoint 5")


    # Drop observations where TESTED is less than 4
    result_df = result_df[result_df['TESTED_total'] >= 2]
    print("checkpoint 6")

    # Fill NaN values with 0
    result_df = result_df.fillna(0)
    print("checkpoint 7")

    cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
    for col in cols:
        result_df[col] = result_df[col].replace('s', 0)
    print("checkpoint 7")
    if 'Regents Common Core Algebra I' in main_data[database]:
        main_data[database]['Regents Common Core Algebra I'] = result_df  # store results in new dictionary instead of main_data
    else:
        main_data[database][new_to_old['Regents Common Core Algebra I']] = result_df

checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7
checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5
checkpoint 6
checkpoint 7
checkpoint 7


In [14]:
# current_df = None
# if 'Regents Common Core English Language Art' in main_data['MAIN2021']:
#     current_df = main_data['MAIN2021']['Regents Common Core English Language Art']
# else:
#     current_df = main_data['MAIN2021'][new_to_old['Regents Common Core English Language Art']]

# # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
# total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == 2021)][['ENTITY_CD', 'TESTED', 'YEAR']]

# # Merge the total students for 'All Students' back into the original DataFrame
# current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))

# # List of subgroups of interest
# KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

# # List to store DataFrames
# df_list = []
# columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
# columns = columns.split()

# # Get ENTITY_NAME for 'All Students' subgroup
# multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]

# # Loop over each subgroup and calculate percentage
# for subgroup in KEPT_SUBGROUPS:
#     temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == 2021)].copy()
#     subgroup = subgroup.upper()
#     temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
#     temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
#     df_list.append(temp_df)

# # Merge all DataFrames on ENTITY_CD
# result_df = multiple_df
# for temp_df in df_list:
#     result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    

# # Drop observations where TESTED is less than 4
# result_df = result_df[result_df['TESTED_total'] >= 2]

# # Fill NaN values with 0
# result_df = result_df.fillna(0)

# cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
# for col in cols:
#     result_df[col] = result_df[col].replace('s', 0)

# main_data['MAIN2021']['Regents Common Core English Language Art'] = result_df  # store results in new dictionary instead of main_data

In [15]:
# current_df = None
# if 'Regents Common Core English Language Art' in main_data['MAIN2022']:
#     current_df = main_data['MAIN2022']['Regents Common Core English Language Art']
# else:
#     current_df = main_data['MAIN2022'][new_to_old['Regents Common Core English Language Art']]

# # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
# total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == 2022)][['ENTITY_CD', 'TESTED', 'YEAR']]

# # Merge the total students for 'All Students' back into the original DataFrame
# current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))

# # List of subgroups of interest
# KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

# # List to store DataFrames
# df_list = []
# columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
# columns = columns.split()

# # Get ENTITY_NAME for 'All Students' subgroup
# multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]

# # Loop over each subgroup and calculate percentage
# for subgroup in KEPT_SUBGROUPS:
#     temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == 2022)].copy()
#     subgroup = subgroup.upper()
#     temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
#     temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
#     df_list.append(temp_df)

# # Merge all DataFrames on ENTITY_CD
# result_df = multiple_df
# for temp_df in df_list:
#     result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    

# # Drop observations where TESTED is less than 4
# result_df = result_df[result_df['TESTED_total'] >= 2]

# # Fill NaN values with 0
# result_df = result_df.fillna(0)

# cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
# for col in cols:
#     result_df[col] = result_df[col].replace('s', 0)

# main_data['MAIN2022']['Regents Common Core English Language Art'] = result_df  # store results in new dictionary instead of main_data

In [16]:
# current_df = None
# if 'Regents Common Core English Language Art' in main_data['MAIN2019']:
#     current_df = main_data['MAIN2019']['Regents Common Core English Language Art']
# else:
#     current_df = main_data['MAIN2019'][new_to_old['Regents Common Core English Language Art']]

# # Filter the DataFrame to only include rows where SUBGROUP_NAME == 'All Students'
# total_students_df = current_df[(current_df['SUBGROUP_NAME'] == 'All Students') & (current_df['YEAR'] == 2019)][['ENTITY_CD', 'TESTED', 'YEAR']]

# # Merge the total students for 'All Students' back into the original DataFrame
# current_df = pd.merge(current_df, total_students_df, on=['ENTITY_CD', 'YEAR'], how='left', suffixes=('', '_total'))

# # List of subgroups of interest
# KEPT_SUBGROUPS = ['Male', 'Female', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian or Native Hawaiian/Other Pacific Islander','Economically Disadvantaged']

# # List to store DataFrames
# df_list = []
# columns = "ENTITY_CD  ENTITY_NAME YEAR SUBJECT TESTED TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF"
# columns = columns.split()

# # Get ENTITY_NAME for 'All Students' subgroup
# multiple_df = current_df[current_df['SUBGROUP_NAME'] == 'All Students'][columns]

# # Loop over each subgroup and calculate percentage
# for subgroup in KEPT_SUBGROUPS:
#     temp_df = current_df[(current_df['SUBGROUP_NAME'] == subgroup) & (current_df['YEAR'] == 2019)].copy()
#     subgroup = subgroup.upper()
#     temp_df[subgroup + '_PCT'] = temp_df['TESTED'] / temp_df['TESTED_total'] * 100
#     temp_df = temp_df[['ENTITY_CD', subgroup + '_PCT']]  # Keep 'ENTITY_CD' in each temp_df
#     df_list.append(temp_df)

# # Merge all DataFrames on ENTITY_CD
# result_df = multiple_df
# for temp_df in df_list:
#     result_df = result_df.merge(temp_df, on='ENTITY_CD', how='outer')
    

# # Drop observations where TESTED is less than 4
# result_df = result_df[result_df['TESTED_total'] >= 2]

# # Fill NaN values with 0
# result_df = result_df.fillna(0)

# cols = "NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF PER_PROF".split()
# for col in cols:
#     result_df[col] = result_df[col].replace('s', 0)

# main_data['MAIN2019'][new_to_old['Regents Common Core English Language Art']] = result_df  # store results in new dictionary instead of main_data

### Concatenate All Dataframes

In [17]:
final_data = pd.concat([
    main_data['MAIN2017'][new_to_old['Regents Common Core Algebra I']],
    main_data['MAIN2018'][new_to_old['Regents Common Core Algebra I']],
    main_data['MAIN2019'][new_to_old['Regents Common Core Algebra I']], 
    main_data['MAIN2021']['Regents Common Core Algebra I'], 
    main_data['MAIN2022']['Regents Common Core Algebra I'], 
])


In [18]:
final_data['SUBJECT'] = 'Regents Common Core Algebra I'
final_data

,ENTITY_CD,ENTITY_NAME,YEAR,SUBJECT,TESTED,TESTED_total,NUM_LEVEL1,PER_LEVEL1,NUM_LEVEL2,PER_LEVEL2,NUM_LEVEL3,PER_LEVEL3,NUM_LEVEL4,PER_LEVEL4,NUM_LEVEL5,PER_LEVEL5,NUM_PROF,PER_PROF,MALE_PCT,FEMALE_PCT,WHITE_PCT,HISPANIC OR LATINO_PCT,BLACK OR AFRICAN AMERICAN_PCT,ASIAN OR NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER_PCT,ECONOMICALLY DISADVANTAGED_PCT
2,522001040003,STILLWATER MIDDLE SCHOOL HIGH SCHOOL,2017,Regents Common Core Algebra I,92,92.0,6,07,4,04,36,39,24,26,22,24,82.0,89.0,55.434783,44.565217,95.652174,0.000000,2.173913,1.086957,18.478261
5,141901060008,LANCASTER HIGH SCHOOL,2017,Regents Common Core Algebra I,385,385.0,9,02,19,05,139,36,136,35,82,21,357.0,92.0,58.441558,41.558442,89.870130,5.194805,1.558442,1.038961,28.831169
8,580224030008,PATCHOGUE-MEDFORD HIGH SCHOOL,2017,Regents Common Core Algebra I,545,545.0,23,04,63,12,312,57,116,21,31,06,459.0,84.0,51.743119,48.256881,42.385321,45.871560,8.256881,2.018349,66.055046
11,10100010034,ALBANY HIGH SCHOOL,2017,Regents Common Core Algebra I,772,772.0,148,19,190,25,358,46,53,07,23,03,434.0,56.0,51.943005,48.056995,14.896373,16.321244,54.792746,10.362694,56.865285
14,10100860907,GREEN TECH HIGH CHARTER SCHOOL,2017,Regents Common Core Algebra I,107,107.0,7,07,21,20,66,62,11,10,2,02,79.0,74.0,100.000000,0.000000,0.934579,14.953271,74.766355,0.934579,62.616822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,331400011586,LYONS COMMUNITY SCHOOL,2022,Regents Common Core Algebra I,38,38.0,25,66,8,21,5,13,0,00,0,00,5,13,28.947368,71.052632,0.000000,63.157895,36.842105,0.000000,94.736842
2441,331500860953,SUMMIT ACADEMY CHARTER SCHOOL,2022,Regents Common Core Algebra I,18,18.0,6,33,6,33,6,33,0,00,0,00,6,33,55.555556,44.444444,0.000000,55.555556,44.444444,0.000000,100.000000
2442,342500011252,QUEENS SCHOOL OF INQUIRY,2022,Regents Common Core Algebra I,80,80.0,12,15,14,18,45,56,8,10,1,01,54,68,55.000000,45.000000,13.750000,23.750000,12.500000,48.750000,70.000000
2443,343000860822,RENAISSANCE CHARTER SCHOOL,2022,Regents Common Core Algebra I,68,68.0,11,16,24,35,30,44,3,04,0,00,33,49,48.529412,51.470588,4.411765,72.058824,4.411765,19.117647,80.882353


In [19]:
# Convert 'YEAR' in final_data to int
final_data['YEAR'] = final_data['YEAR'].astype(int)

# Select specific columns from dropout_df
dropout_subset = dropout_df[['aggregation_code', 'report_school_year', 'dropout_pct']]

# Merge dropout_df with final_data
final_data = pd.merge(final_data, dropout_subset, left_on=['ENTITY_CD', 'YEAR'], right_on=['aggregation_code', 'report_school_year'], how='left')

# Replace '-' with np.nan
final_data['dropout_pct'] = final_data['dropout_pct'].replace('-', np.nan)

# Remove '%' from 'dropout_pct' and convert to float
final_data['dropout_pct'] = final_data['dropout_pct'].str.rstrip('%').astype('float')

final_data['dropout_pct'] = final_data['dropout_pct'].fillna(0)

In [20]:
final_data

,ENTITY_CD,ENTITY_NAME,YEAR,SUBJECT,TESTED,TESTED_total,NUM_LEVEL1,PER_LEVEL1,NUM_LEVEL2,PER_LEVEL2,NUM_LEVEL3,PER_LEVEL3,NUM_LEVEL4,PER_LEVEL4,NUM_LEVEL5,PER_LEVEL5,NUM_PROF,PER_PROF,MALE_PCT,FEMALE_PCT,WHITE_PCT,HISPANIC OR LATINO_PCT,BLACK OR AFRICAN AMERICAN_PCT,ASIAN OR NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER_PCT,ECONOMICALLY DISADVANTAGED_PCT,aggregation_code,report_school_year,dropout_pct
0,522001040003,STILLWATER MIDDLE SCHOOL HIGH SCHOOL,2017,Regents Common Core Algebra I,92,92.0,6,07,4,04,36,39,24,26,22,24,82.0,89.0,55.434783,44.565217,95.652174,0.000000,2.173913,1.086957,18.478261,NaN,NaN,0.0
1,141901060008,LANCASTER HIGH SCHOOL,2017,Regents Common Core Algebra I,385,385.0,9,02,19,05,139,36,136,35,82,21,357.0,92.0,58.441558,41.558442,89.870130,5.194805,1.558442,1.038961,28.831169,NaN,NaN,0.0
2,580224030008,PATCHOGUE-MEDFORD HIGH SCHOOL,2017,Regents Common Core Algebra I,545,545.0,23,04,63,12,312,57,116,21,31,06,459.0,84.0,51.743119,48.256881,42.385321,45.871560,8.256881,2.018349,66.055046,NaN,NaN,0.0
3,10100010034,ALBANY HIGH SCHOOL,2017,Regents Common Core Algebra I,772,772.0,148,19,190,25,358,46,53,07,23,03,434.0,56.0,51.943005,48.056995,14.896373,16.321244,54.792746,10.362694,56.865285,NaN,NaN,0.0
4,10100860907,GREEN TECH HIGH CHARTER SCHOOL,2017,Regents Common Core Algebra I,107,107.0,7,07,21,20,66,62,11,10,2,02,79.0,74.0,100.000000,0.000000,0.934579,14.953271,74.766355,0.934579,62.616822,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5752,331400011586,LYONS COMMUNITY SCHOOL,2022,Regents Common Core Algebra I,38,38.0,25,66,8,21,5,13,0,00,0,00,5,13,28.947368,71.052632,0.000000,63.157895,36.842105,0.000000,94.736842,3.314000e+11,2022.0,4.0
5753,331500860953,SUMMIT ACADEMY CHARTER SCHOOL,2022,Regents Common Core Algebra I,18,18.0,6,33,6,33,6,33,0,00,0,00,6,33,55.555556,44.444444,0.000000,55.555556,44.444444,0.000000,100.000000,3.315009e+11,2022.0,6.0
5754,342500011252,QUEENS SCHOOL OF INQUIRY,2022,Regents Common Core Algebra I,80,80.0,12,15,14,18,45,56,8,10,1,01,54,68,55.000000,45.000000,13.750000,23.750000,12.500000,48.750000,70.000000,3.425000e+11,2022.0,3.0
5755,343000860822,RENAISSANCE CHARTER SCHOOL,2022,Regents Common Core Algebra I,68,68.0,11,16,24,35,30,44,3,04,0,00,33,49,48.529412,51.470588,4.411765,72.058824,4.411765,19.117647,80.882353,3.430009e+11,2022.0,2.0


# Import Virtual Mode Data

In [21]:
virtual = pd.read_csv("../../data/New_York_Schools_LearningModelData_Final.csv", thousands=',')
virtual['Charter'] = virtual['Charter'].replace({'Yes': 1, 'No': 0})

In [22]:
virtual = virtual[virtual['TimePeriodStart'].str.endswith(('21', '22'))]
virtual.head()

,StateName,StateAbbrev,DataLevel,Charter,SchoolName,SchoolType,NCESSchoolID,StateAssignedSchoolID,DistrictName,DistrictType,NCESDistrictID,StateAssignedDistrictID,TimePeriodInterval,TimePeriodStart,TimePeriodEnd,EnrollmentTotal,LearningModel,LearningModelGrK5,LearningModelGr68,LearningModelGr912,LearningModelStateCat,LearningModelStateCatGrK5,LearningModelStateCatGr68,LearningModelStateCatGr912,EnrollmentInPerson,EnrollmentHybrid,EnrollmentVirtual,StaffCount,StaffCountInPerson
12,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/3/21,1/9/21,378,Virtual,NaN,NaN,NaN,Remote Only,NaN,NaN,NaN,79.0,0.0,299.0,81.0,45.0
13,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/10/21,1/16/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,237.0,141.0,81.0,81.0
14,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/17/21,1/23/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,226.0,152.0,81.0,76.0
15,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/24/21,1/30/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,229.0,149.0,81.0,81.0
16,New York,NY,School,0,James A Green High School,Regular school,360000104498,211003040002,Dolgeville Central School District,Regular local school district,3600001,211003040000,Weekly,1/31/21,2/6/21,378,Hybrid,NaN,NaN,NaN,Hybrid,NaN,NaN,NaN,0.0,230.0,148.0,81.0,80.0


In [23]:
# Convert the date columns to datetime format.
virtual['TimePeriodStart'] = pd.to_datetime(virtual['TimePeriodStart'])
virtual['TimePeriodEnd'] = pd.to_datetime(virtual['TimePeriodEnd'])

# Create a new column for year
virtual['YEAR'] = virtual['TimePeriodStart'].dt.year

# Fill in any missing values in LearningModel with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].fillna('InPerson')

# Replace 'In-person' with 'InPerson'
virtual['LearningModel'] = virtual['LearningModel'].replace('In-person', 'InPerson')

# Calculate the number of days for each row
virtual['Days'] = (virtual['TimePeriodEnd'] - virtual['TimePeriodStart']).dt.days

# Group by School, Year, LearningModel, and Charter and sum the number of days
grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel', 'Charter'])['Days'].sum().reset_index()

# Pivot the data so we have separate columns for each learning model
pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR', 'Charter'], columns='LearningModel', values='Days', fill_value=0)

# # Group by School, Year, and LearningModel and sum the number of days
# grouped = virtual.groupby(['StateAssignedSchoolID', 'YEAR', 'LearningModel'])['Days'].sum().reset_index()

# # Pivot the data so we have separate columns for each learning model
# pivot = grouped.pivot_table(index=['StateAssignedSchoolID', 'YEAR'], columns='LearningModel', values='Days', fill_value=0)

# Reset the index
pivot.reset_index(inplace=True)

# Calculate the total days in each year
pivot['TotalDays'] = pivot['Virtual'] + pivot['Hybrid'] + pivot['InPerson']

# Calculate the percentage of days that are virtual and hybrid for each year
pivot['VirtualPercent'] = pivot['Virtual'] / pivot['TotalDays']
pivot['HybridPercent'] = pivot['Hybrid'] / pivot['TotalDays']

# Calculate the score for each year
pivot['Score'] = (pivot['Virtual'] + 0.5 * pivot['Hybrid']) / pivot['TotalDays']

# Reset the column names after pivot
pivot.columns.name = None

In [24]:
pivot = pivot.drop(columns=['InPerson', 'Hybrid', 'Virtual', 'TotalDays'])
pivot

,StateAssignedSchoolID,YEAR,Charter,VirtualPercent,HybridPercent,Score
0,10100010014,2021,0,0.04,0.88,0.48
1,10100010016,2021,0,0.04,0.88,0.48
2,10100010018,2021,0,0.04,0.88,0.48
3,10100010019,2021,0,0.04,0.88,0.48
4,10100010023,2021,0,0.04,0.88,0.48
...,...,...,...,...,...,...
4398,680601060001,2021,0,0.08,0.00,0.08
4399,680601060002,2021,0,0.08,0.00,0.08
4400,680601060005,2021,0,0.08,0.00,0.08
4401,680801040001,2021,0,0.08,0.00,0.08


In [25]:
# Merge dropout_df with final_data
final_data = pd.merge(final_data, pivot, left_on=['ENTITY_CD', 'YEAR'], right_on=['StateAssignedSchoolID', 'YEAR'], how='left')



In [26]:
final_data = final_data.drop(columns=[col for col in "ENTITY_NAME TESTED_total NUM_LEVEL1 PER_LEVEL1 NUM_LEVEL2 PER_LEVEL2 NUM_LEVEL3 PER_LEVEL3 NUM_LEVEL4 PER_LEVEL4 NUM_LEVEL5 PER_LEVEL5 NUM_PROF aggregation_code report_school_year StateAssignedSchoolID MALE_PCT FEMALE_PCT dropout_pct SUBJECT".split()])

In [27]:
# Rename multiple columns
final_data = final_data.rename(columns={'ENTITY_CD': 'schoolcode', 
                        'TESTED': 'totalenroll', 
                        'PER_PROF': 'mathpass', 
                        'WHITE_PCT': 'white',
                        'HISPANIC OR LATINO_PCT': 'hispanic',     
                        'BLACK OR AFRICAN AMERICAN_PCT': 'black',         
                        'ASIAN OR NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER_PCT': 'asian',                     
                        'ECONOMICALLY DISADVANTAGED_PCT': 'lowincome',                                         
                        'VirtualPercent': 'virtualper',    
                        'HybridPercent': 'hybridper',    
                        'Score': 'schoolmode',    
                        'YEAR': 'year',    
                        'Charter': "charter"
                                       })

In [28]:
# Identify 'schoolcode' values of rows in 2021 where 'schoolmode' is NaN
schoolcodes_to_remove = final_data.loc[(final_data['year'] == 2021) & (final_data['schoolmode'].isna()), 'schoolcode'].unique()

# Remove all rows with those 'schoolcode' values
final_data = final_data.loc[~final_data['schoolcode'].isin(schoolcodes_to_remove)]

In [29]:
final_data = final_data.fillna(0)
final_data['district'] = final_data['schoolcode'].astype(str).str[-6:-4].astype(int)
final_data['county'] = final_data['schoolcode'].astype(str).str[-2:]

final_cols = [
'schoolcode',
'district',
'county',
 'year',
 'charter',
 'mathpass',
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenroll',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]


In [30]:
final_data = final_data[final_cols] 

In [31]:
final_data = final_data[final_cols]
rounding_cols = [
 'schoolmode',
 'virtualper',
 'hybridper',
 'totalenroll',
 'lowincome',
 'white',
 'black',
 'hispanic',
 'asian',
      ]

final_data[rounding_cols] = final_data[rounding_cols].round(4)

In [32]:
# # Sort the data by school and year
# final_data.sort_values(['schoolcode', 'year'], inplace=True)

# # Replace 0 with NaN in 'district' column
# final_data['district'].replace(0, pd.NA, inplace=True)

# # Now apply the 'ffill' and 'bfill' methods
# final_data['district'] = final_data.groupby('schoolcode')['district'].apply(lambda group: group.ffill().bfill())

# final_data = final_data.dropna(subset=['district'])

In [33]:
final_data_sorted = final_data.sort_values(by='schoolcode')
final_data_sorted = final_data_sorted.reset_index(drop=True)
final_data_sorted.head(20)

,schoolcode,district,county,year,charter,mathpass,schoolmode,virtualper,hybridper,totalenroll,lowincome,white,black,hispanic,asian
0,10100010034,1,34,2019,0.0,56,0.00,0.00,0.00,732,72.5410,16.5301,55.3279,16.5301,8.0601
1,10100010034,1,34,2022,0.0,34,0.00,0.00,0.00,439,74.9431,11.3895,46.4692,22.0957,12.9841
2,10100010034,1,34,2017,0.0,56.0,0.00,0.00,0.00,772,56.8653,14.8964,54.7927,16.3212,10.3627
3,10100010034,1,34,2018,0.0,49,0.00,0.00,0.00,630,73.3333,17.9365,56.0317,15.3968,7.4603
4,10100010034,1,34,2021,0.0,60,0.48,0.04,0.88,10,60.0000,30.0000,40.0000,20.0000,0.0000
5,10100860960,86,60,2017,0.0,61.0,0.00,0.00,0.00,168,82.7381,5.9524,71.4286,13.0952,4.1667
6,10100860960,86,60,2018,0.0,60,0.00,0.00,0.00,188,86.1702,11.1702,63.2979,13.8298,5.3191
7,10100860960,86,60,2022,0.0,23,0.00,0.00,0.00,80,68.7500,7.5000,57.5000,25.0000,6.2500
8,10100860960,86,60,2019,0.0,41,0.00,0.00,0.00,155,86.4516,9.0323,64.5161,15.4839,5.1613
9,10201040001,4,01,2019,0.0,83,0.00,0.00,0.00,52,40.3846,94.2308,0.0000,0.0000,0.0000


# Export NYC Data

In [34]:
first_export = final_data[final_data["year"] != 2022]

first_export.to_csv("../final_data_components/data_component_nyc_math.csv")

In [35]:
# final_data.to_csv("for_running.csv")